In [1]:
import requests
import time
import json
import threading
import queue
import random
import datetime
import os
import urllib3

from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from collections import defaultdict
from pprint import pprint

from config.assistant import *

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
#from config.site_config import *

In [2]:
# searchType=search
# searchword=컴퓨터비전         # 검색어
# loc_mcd=101000,102000         # 지역, 101000(서울),102000(경기)
# company_cd=0,1,2,3,4,5,6,7    # 채용제공구분 / 9(파견/대행), 10(헤드헌팅)
# search_optional_item=y
# search_done=y
# panel_count=y
# recruitPage=1                 # 현재 페이지뷰
# recruitSort=relation          # 정렬방법
# recruitPageCount=100          # 표시 개수
# inner_com_type=
# show_applied=                 # 지원한 공고 표시
# quick_apply=                  # 빠른 입사지원
# except_read=                  # 조회한 페이지 제외
# ai_head_hunting=
# mainSearch=n

In [6]:
search_words = ["컴퓨터비전", "OPENCV"]
page_view_items = 100
company_cd='0,1,2,3,4,5,6,7'
base_link = 'https://www.saramin.co.kr'
job_group = {}
item_count = 0

In [7]:

# 저장된 키워드 별 검색
for search_word in search_words:
    tic()
    current_page = 1
    
    # 검색 결과 리스트 페이지 별 확인
    search_loop = True
    while(search_loop):
        search_link = f'https://www.saramin.co.kr/zf_user/search/recruit'\
            + f'?search_area=main'\
            + f'&search_done=y'\
            + f'&search_optional_item=n'\
            + f'&searchType=search'\
            + f'&recruitSort=relation'\
            + f'&searchword={search_word}'\
            + f'&recruitPage={current_page}'\
            + f'&recruitPageCount={page_view_items}'\
            + f'&company_cd={company_cd}'\
            + f'&mainSearch=y'

        response = requests.get(search_link, verify=False)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        if len(soup.find_all('div', attrs={'class': 'info_no_result'})) == 1:
            search_loop = False
        else:
            current_page += 1

            page_items = len(soup.find_all('h2', attrs={'class': 'job_tit'}))

            # 해당 페이지 리스트 (1 ~ 100)
            for i in range(1, page_items+1):
                elements = soup.select(f'div.content > div:nth-child({i})')[0]

                # 채용공고명
                str_title = elements.find('a', attrs={'class': 'data_layer'})['title']

                # 회사명
                str_company = elements.find('a', attrs={'class': 'track_event data_layer'}).string
                str_company = str_company.replace(' ', '').replace('\n', '').replace('(주)', '')

                # 채용공고 링크
                str_title_link = base_link + elements.find('a', attrs={'class': 'data_layer'})['href']

                # 회사 링크
                str_company_link = base_link + elements.find('a', attrs={'class': 'track_event data_layer'})['href']

                # Dictionary에 저장
                if str_company in job_group:    # 회사명 - 기존에 있으면
                    for company in job_group[str_company]:
                        if company['title'] == str_title:    # 채용공고명 - 기존에 있으면
                            break
                    else:
                        job_group[str_company].append({'title': str_title, \
                                                       'title_link': str_title_link, \
                                                       'company_link': str_company_link \
                                                      })

                else:    # 회사명 - 신규진입
                    job_group[str_company] = []
                    job_group[str_company].append({'title': str_title, \
                                                   'title_link': str_title_link, \
                                                   'company_link': str_company_link \
                                                  })

                item_count += 1
    toc()

Elapsed time is 63.420079946517944 seconds.
Elapsed time is 13.11435317993164 seconds.
